In [ ]:
import kfp
from kfp import dsl
from kfp import kubernetes
from kfp.dsl import Input, Output, Dataset, component

# --- 组件 1: 读取源文件并排序 ---
@dsl.component(
    base_image="python:3.10",
    packages_to_install=["pandas","kfp","kfp-kubernetes"],
)
def sort_csv_component(
        input_file_path: str,        # 输入：源文件在 PVC 中的绝对路径
        sorted_output: Output[Dataset] # 输出：排序后的数据（KFP 自动管理的临时路径）
):
    import os

    # 1. 检查文件是否存在
    if not os.path.exists(input_file_path):
        raise FileNotFoundError(f"找不到源文件: {input_file_path}")

    # 2. 读取数据并转换
    with open(input_file_path, 'r', encoding='utf-8') as f:
        # 去重、去空行并转为整数
        numbers = [int(line.strip()) for line in f if line.strip()]

    # 3. 降序排序
    numbers.sort(reverse=True)

    # 4. 将结果写入 KFP 自动生成的临时输出路径 (.path)
    with open(sorted_output.path, 'w', encoding='utf-8') as f:
        for num in numbers:
            f.write(f"{num}\n")

# --- 组件 2: 打印并保存到 PVC ---
@dsl.component(
    base_image="python:3.10",
    packages_to_install=["pandas","kfp","kfp-kubernetes"],
)
def save_and_print_component(
        sorted_data_input: Input[Dataset], # 接收上一步产生的 Artifact
        pvc_output_directory: str,        # 目标 PVC 目录
        file_name: str                     # 最终保存的文件名
):
    import os
    import shutil

    # 1. 创建目标目录（如果不存在）
    if not os.path.exists(pvc_output_directory):
        os.makedirs(pvc_output_directory)

    target_path = os.path.join(pvc_output_directory, file_name)

    # 2. 使用 shutil 高效复制文件
    # 从 KFP 的临时存储（sorted_data_input.path）拷贝到指定的 PVC 路径
    shutil.copy(sorted_data_input.path, target_path)

    # 3. 打印结果到日志以便查看
    with open(target_path, 'r') as f:
        print("--- 排序后的结果内容 ---")
        print(f.read())

    print(f"成功保存到 PVC 路径: {target_path}")

# --- 定义 Pipeline 工作流 ---
@dsl.pipeline(
    name="csv-sort-and-save-to-pvc",
    description="从 PVC 读取数据，排序后写回 PVC"
)
def demo_sort_pipeline(
        raw_input_path: str = "/mnt/shared/input_numbers.csv",
        pvc_dir: str = "/mnt/shared/output",
        final_name: str = "sorted_numbers.csv"
):
    # 第一步任务
    task1 = sort_csv_component(input_file_path=raw_input_path)

    # 第二步任务（传入第一步的输出结果）
    task2 = save_and_print_component(
        sorted_data_input=task1.outputs['sorted_output'],
        pvc_output_directory=pvc_dir,
        file_name=final_name
    )

    # --- 为所有步骤挂载同一个 PVC ---
    # 假设你已有一个名为 'data-pvc' 的 PVC
    tasks = [task1, task2]
    for task in tasks:
        kubernetes.mount_pvc(
            task,
            pvc_name="aidatasource",
            mount_path="/mnt/shared" # 容器内访问该 PVC 的根目录
        )

# 编译pipeline
import kfp
from kfp.client import Client
# 编译成 YAML 供上传使用
# --- 配置 ---
NAMESPACE = "kubeflow-user-example-com"
PIPELINE_NAME = "demo_sort_pipeline" # 在 UI 中显示的名称
PIPELINE_YAML = "demo_sort_pipeline.yaml"
kfp.compiler.Compiler().compile(demo_sort_pipeline, PIPELINE_YAML)

# 运行Pipeline
## run
run_result = client.create_run_from_pipeline_func(
    demo_sort_pipeline,
    arguments={
        'raw_input_path': '/mnt/shared/input_numbers.csv',
        'pvc_dir': '/mnt/shared/output',
        'final_name': 'sorted_numbers.csv'
    },
    namespace=NAMESPACE,
    experiment_name="demo_sort_pipeline_test_experiment",
    enable_caching=False,
)

print(f"任务已提交！运行 ID: {run_result.run_id}")
